In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs

1. 날짜를 순회하며 크롤링을 진행하기에 URL이 상대적으로 더 편하게 구성되어 있어서 Naver뉴스 사용
2. Naver뉴스의 class='_article_body_contents'인 div태그에 본문 내용이 포함되어 있음을 확인
3. 크롤링 과정에서 제거해야 할 정보들 확인
 - 신문사마다 \<a\>태그에 기사 내용과 상관없는 링크를 포함시켜 놓음
 - 영어신문(코리아헤럴드) 제외
 - 저작권 명시하는 문장(제일 하단에 ⓒ표시를 포함하고 있음)
 - 기자의 이메일(당연히 @를 포함하는 문자열로 식별가능)
 - 괄호에 둘러쌓여 부수적인 정보를 제공하는 기타 내용들(신문사 이름 등)
 
이 노트북에서 예시로 한 기사의 본문을 추출해 보려고 한다.

In [2]:
response = requests.get("https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=016&aid=0001719948")
response.status_code

200

## 1. 코리아헤럴드 제외

In [3]:
soup = bs(response.text, 'html.parser')
publisher = soup.select('div.press_logo img')[0]['title']
if publisher != '코리아헤럴드':
    print(publisher)

헤럴드경제


## 2. \<a\>태그 제거

In [4]:
body = soup.find('div', attrs={'class':'_article_body_contents'})
for a_tag in body.select('a'):
    a_tag.extract()
print(body.prettify())

<div class="_article_body_contents" id="articleBodyContents">
 <!-- 본문 내용 -->
 <!-- TV플레이어 -->
 <!-- // TV플레이어 -->
 <script type="text/javascript">
  // flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
 </script>
 <strong class="media_end_summary">
  日, 비자 발급제한 조치 재연장
  <br/>
  라오스엔 전세기투입 불투명
 </strong>
 국내 코로나19 재확산세로 인해 우리 국민의 해외 이동 보장을 확대해 가던 정부의 노력에 제동이 걸렸다. 기업인 신속 통로(패스트트랙) 추가 협상이 늦어지는 데다가 한국인에 대한 비자 발급 문제와 전세기 투입에도 어려움을 겪고 있다.
 <br/>
 <br/>
 2일 주한일본대사관 등에 따르면 일본 외무성은 지난달 31일까지였던 한국인에 대한 검역 강화와 비자 발급 제한 조치를 당분간 연장하기로 했다.
 <br/>
 <br/>
 그간 18개국에서 1만8000여 명의 기업인 예외입국을 도운 정부는 10여 개 국과 간소화된 입국 절차를 내용으로 하는 패스트트랙 도입 협상을 계속하고 있지만, 최근 국내 코로나19 재확산으로 협상에 어려움을 겪고 있다.
 <br/>
 <br/>
 해외 코로나19 재확산 때문에 모처럼 뚫렸던 우리 교민의 이동이 다시 제한되는 경우도 생기고 있다. 당장 라오스는 지난달 31일 공지를 통해 기존 코로나19 대응조치 연장을 결정했다. 이 때문에 당장 이번 달로 예정됐던 우리 교민 이송을 위한 전세기편 투입 일정이 불투명해졌다.
 <br/>
 <br/>
 주라오스 한국대사관은 “우리 국민들의 긴급한 용무 및 기업 활동 등 다양한 수요를 고려하여 9월 중 전세기가 운항될 수 있도록 라오스 정부와 최대한 협의할 예정”이라고 설명했다. 유오상 기자
 <br/>
 <br/>
 <br/>
 <br

## 3. 카피라이트 문장 및 이메일 제거 후 괄호로 싸인 문자열 제거

In [5]:
def extractor(body):
    tags = [tag for tag in body]
    types = [type(tag) for tag in body]
    
    text = ' '.join([tag.strip() for tag, type_ in zip(tags, types) if 
     str(type_) == "<class 'bs4.element.NavigableString'>" and
     '@' not in tag and
     'ⓒ' not in tag
    ]).strip()
    
    pattern = re.compile('[\(\[].{,20}[\)\]]')
    return pattern.sub('', text)

괄호로 싸인 문자열을 제거하는 과정에서 괄호에 싸인 글자 수를 최대 20까지로 지정했다. 이는 기사 초반에 있는 여는 괄호가 기사 말미에 있는 닫는 괄호와 한 쌍인 것으로 인식되서 기사 전체가 다 사라지는 불상사를 방지하기 위해서이다. 결과를 확인하면 아래와 같다.

In [6]:
extractor(body)

'국내 코로나19 재확산세로 인해 우리 국민의 해외 이동 보장을 확대해 가던 정부의 노력에 제동이 걸렸다. 기업인 신속 통로 추가 협상이 늦어지는 데다가 한국인에 대한 비자 발급 문제와 전세기 투입에도 어려움을 겪고 있다. 2일 주한일본대사관 등에 따르면 일본 외무성은 지난달 31일까지였던 한국인에 대한 검역 강화와 비자 발급 제한 조치를 당분간 연장하기로 했다. 그간 18개국에서 1만8000여 명의 기업인 예외입국을 도운 정부는 10여 개 국과 간소화된 입국 절차를 내용으로 하는 패스트트랙 도입 협상을 계속하고 있지만, 최근 국내 코로나19 재확산으로 협상에 어려움을 겪고 있다. 해외 코로나19 재확산 때문에 모처럼 뚫렸던 우리 교민의 이동이 다시 제한되는 경우도 생기고 있다. 당장 라오스는 지난달 31일 공지를 통해 기존 코로나19 대응조치 연장을 결정했다. 이 때문에 당장 이번 달로 예정됐던 우리 교민 이송을 위한 전세기편 투입 일정이 불투명해졌다. 주라오스 한국대사관은 “우리 국민들의 긴급한 용무 및 기업 활동 등 다양한 수요를 고려하여 9월 중 전세기가 운항될 수 있도록 라오스 정부와 최대한 협의할 예정”이라고 설명했다. 유오상 기자'